# 🏝️ Asistente Turístico de Tenerife

**Práctica Final - Large Language Models**

**Autor:** Antonio Rodriguez

---

## Índice

1. Configuración del Entorno
2. Carga de Datos y Vector Store
3. RAG Chain con Function Calling
4. Demostración de Funcionalidades
5. Evaluación y Análisis
6. Conclusiones

---

## 1. Configuración del Entorno

In [1]:
# Importar módulos y configuración
from src.conf import OPENAI_API_KEY, MODEL_CONFIG, PDF_PATH, CHUNK_SIZE, CHUNK_OVERLAP
from src.api_client import OpenAIClient
from src.data_loader import DataLoader
from src.vector_store import VectorStore
from src.rag_chain import RAGChain
from src.weather_service import WeatherService

print("✅ Módulos importados")
print(f"\n📋 Configuración del modelo:")
for key, value in MODEL_CONFIG.items():
    print(f"   {key}: {value}")
print(f"\n📋 Configuración RAG:")
print(f"   chunk_size: {CHUNK_SIZE}")
print(f"   chunk_overlap: {CHUNK_OVERLAP}")

✅ Módulos importados

📋 Configuración del modelo:
   model: gpt-4o-mini
   temperature: 0.3
   max_tokens: 1024
   top_p: 0.9

📋 Configuración RAG:
   chunk_size: 1000
   chunk_overlap: 200


In [2]:
# Inicializar cliente OpenAI
client = OpenAIClient()
print("✅ OpenAI Client:", "Conectado" if client.test_connection() else "Error")

2026-01-22 14:33:04,203 - INFO - OpenAI client initialized with model: gpt-4o-mini
2026-01-22 14:33:05,370 - INFO - OpenAI connection test successful


✅ OpenAI Client: Conectado


---

## 2. Carga de Datos y Vector Store

In [3]:
# Cargar PDF
loader = DataLoader(PDF_PATH)
pages = loader.load()
stats = loader.get_stats()

print("✅ Documento cargado:")
print(f"   Páginas: {stats['num_pages']}")
print(f"   Caracteres: {stats['total_chars']:,}")
print(f"   Palabras: {stats['total_words']:,}")

2026-01-22 14:33:13,457 - INFO - DataLoader initialized with: data/TENERIFE.pdf
2026-01-22 14:33:13,779 - INFO - Loaded 25 pages from data/TENERIFE.pdf


✅ Documento cargado:
   Páginas: 25
   Caracteres: 16,094
   Palabras: 2,817


In [4]:
# Construir vector store
vector_store = VectorStore()
num_chunks = vector_store.build_from_documents(pages)
chunk_stats = vector_store.get_chunk_stats()

print("✅ Vector Store construido:")
print(f"   Chunks creados: {chunk_stats['num_chunks']}")
print(f"   Tamaño promedio: {chunk_stats['avg_chunk_size']} chars")
print(f"   Tamaño mínimo: {chunk_stats['min_chunk_size']} chars")
print(f"   Tamaño máximo: {chunk_stats['max_chunk_size']} chars")

2026-01-22 14:33:23,037 - INFO - VectorStore initialized (chunk_size=1000, overlap=200)
2026-01-22 14:33:23,039 - INFO - Created 30 chunks from 25 pages
2026-01-22 14:33:24,074 - INFO - Vector store built and persisted to ./chroma_db


✅ Vector Store construido:
   Chunks creados: 30
   Tamaño promedio: 570 chars
   Tamaño mínimo: 86 chars
   Tamaño máximo: 987 chars


---

## 3. RAG Chain con Function Calling

In [5]:
# Inicializar servicios
weather_service = WeatherService(simulated=True)
rag = RAGChain(client, vector_store, weather_service=weather_service, max_history=5)

print("✅ Weather Service: Inicializado (modo simulado)")
print("✅ RAG Chain: Inicializado con function calling")

2026-01-22 14:33:33,668 - INFO - WeatherService initialized (simulated=True)
2026-01-22 14:33:33,671 - INFO - RAGChain initialized (max_history=5, tools=1)


✅ Weather Service: Inicializado (modo simulado)
✅ RAG Chain: Inicializado con function calling


---

## 4. Demostración de Funcionalidades

### 4.1 RAG - Consultas sobre la guía

In [6]:
rag.clear_history()

result = rag.query("¿Qué puedo visitar en La Laguna?", k=5)
print("👤 Usuario:", result["question"])
print("\n🤖 Asistente:", result["answer"])
print(f"\n📚 Fuentes: {[s['page'] for s in result['sources']]}")
print(f"🔧 Tool called: {result.get('tool_called', False)}")

2026-01-22 14:33:38,468 - INFO - Conversation history cleared
2026-01-22 14:33:38,762 - INFO - Search query: '¿Qué puedo visitar en La Laguna?...' returned 5 results
2026-01-22 14:33:38,763 - INFO - Retrieved 5 chunks for: ¿Qué puedo visitar en La Laguna?...
2026-01-22 14:33:44,378 - INFO - Completion with tools - finish_reason: stop
2026-01-22 14:33:44,380 - INFO - Generated answer (no tool call): 801 chars


👤 Usuario: ¿Qué puedo visitar en La Laguna?

🤖 Asistente: En La Laguna, que es Patrimonio de la Humanidad, hay varios lugares de interés que puedes visitar:

1. **Plaza de la Iglesia de la Concepción**: Este es un punto central en La Laguna y es conocido por su hermosa iglesia. Es un lugar ideal para comenzar tu recorrido.

2. **Centro de La Laguna**: Pasear por el centro te permitirá disfrutar de los edificios de estilo colonial que caracterizan la ciudad. Es un lugar perfecto para callejear y descubrir la arquitectura local.

3. **La Cueva de Casiano**: Si decides tomar algo o comer, este restaurante es muy recomendado. Prueba la batata con almogrote y deja que te recomienden otros platos.

Te sugiero que aparques cerca de la Plaza de la Concepción y disfrutes de un paseo por el centro. ¡No olvides disfrutar de la atmósfera única de esta ciudad!

📚 Fuentes: [3, 2, 10, 4, 11]
🔧 Tool called: False


### 4.2 Function Calling - Consultas de clima

In [7]:
rag.clear_history()

result = rag.query("¿Qué tiempo hará mañana en Tenerife?")
print("👤 Usuario:", result["question"])
print("\n🤖 Asistente:", result["answer"])
print(f"\n🔧 Tool called: {result.get('tool_called', False)}")

2026-01-22 14:33:52,067 - INFO - Conversation history cleared
2026-01-22 14:33:52,392 - INFO - Search query: '¿Qué tiempo hará mañana en Tenerife?...' returned 3 results
2026-01-22 14:33:52,393 - INFO - Retrieved 3 chunks for: ¿Qué tiempo hará mañana en Tenerife?...
2026-01-22 14:33:53,605 - INFO - Completion with tools - finish_reason: tool_calls
2026-01-22 14:33:53,606 - INFO - Tool call requested: get_weather
2026-01-22 14:33:53,607 - INFO - Tool call arguments: {"date":"mañana"}
2026-01-22 14:33:53,608 - INFO - Weather request: date=2026-01-23, location=Tenerife, simulated=True
2026-01-22 14:33:53,608 - INFO - Weather response: Ventoso, 19°C
2026-01-22 14:33:53,609 - INFO - Tool result: {'date': '2026-01-23', 'location': 'Tenerife', 'temperature_high': 19, 'temperature_low': 17, 'condition': 'Ventoso', 'humidity': 71, 'wind_speed': 14, 'recommendation': 'Cuidado en zonas de acantilados. Evita actividades acuáticas.', 'simulated': True}
2026-01-22 14:33:57,083 - INFO - Completion re

👤 Usuario: ¿Qué tiempo hará mañana en Tenerife?

🤖 Asistente: Mañana en Tenerife se espera un día ventoso, con temperaturas que oscilarán entre los 17°C y 19°C. La humedad será del 71% y la velocidad del viento alcanzará los 14 km/h.

**Recomendaciones:**
- Ten cuidado en zonas de acantilados debido al viento.
- Es mejor evitar actividades acuáticas por las condiciones del mar.

Si tienes planes para explorar la isla, podrías disfrutar de una visita al Teide o pasear por Santa Cruz, donde el clima será más fresco y agradable. ¡Espero que disfrutes tu día en Tenerife!

🔧 Tool called: True


### 4.3 Diálogo Multiturno

In [8]:
rag.clear_history()
print("=" * 60)
print("CONVERSACIÓN MULTITURNO")
print("=" * 60)

# Turno 1
result = rag.query("¿Qué playas hay en el norte de Tenerife?", k=5)
print(f"\n👤 Turno 1: {result['question']}")
print(f"🤖 Respuesta: {result['answer'][:300]}...")

2026-01-22 14:34:07,346 - INFO - Conversation history cleared


CONVERSACIÓN MULTITURNO


2026-01-22 14:34:07,607 - INFO - Search query: '¿Qué playas hay en el norte de Tenerife?...' returned 5 results
2026-01-22 14:34:07,608 - INFO - Retrieved 5 chunks for: ¿Qué playas hay en el norte de Tenerife?...
2026-01-22 14:34:13,789 - INFO - Completion with tools - finish_reason: stop
2026-01-22 14:34:13,790 - INFO - Generated answer (no tool call): 922 chars



👤 Turno 1: ¿Qué playas hay en el norte de Tenerife?
🤖 Respuesta: En el norte de Tenerife, hay varias playas que puedes visitar. Aquí te menciono algunas destacadas:

1. **Playa Martiánez**: Ubicada en Puerto de La Cruz, es una playa que forma parte de un conjunto de piscinas naturales. Es un lugar ideal para disfrutar del mar y relajarte.

2. **Playa de San Telmo...


In [9]:
# Turno 2 - Requiere contexto del turno anterior
result = rag.query("¿Cuál me recomiendas para ver el atardecer?")
print(f"👤 Turno 2: {result['question']}")
print(f"🤖 Respuesta: {result['answer']}")

2026-01-22 14:34:14,077 - INFO - Search query: '¿Cuál me recomiendas para ver el atardecer?...' returned 3 results
2026-01-22 14:34:14,079 - INFO - Retrieved 3 chunks for: ¿Cuál me recomiendas para ver el atardecer?...
2026-01-22 14:34:17,002 - INFO - Completion with tools - finish_reason: stop
2026-01-22 14:34:17,003 - INFO - Generated answer (no tool call): 514 chars


👤 Turno 2: ¿Cuál me recomiendas para ver el atardecer?
🤖 Respuesta: Te recomiendo la **Playa de Benijo** para disfrutar de un atardecer espectacular. Según la guía (página 7), es conocida por sus impresionantes vistas y, si el día está despejado, podrás disfrutar de uno de los atardeceres más bellos de la isla. 

Recuerda que para llegar a esta playa, deberás aparcar cerca y acceder a la playa desde un punto específico, aunque puede que encuentres el acceso cortado con una valla, pero aún así puedes acceder. ¡No te olvides de llevar tu cámara para capturar ese momento mágico!


In [10]:
# Turno 3 - Combina memoria + function calling
result = rag.query("¿Qué tiempo hará mañana para ir allí?")
print(f"👤 Turno 3: {result['question']}")
print(f"🤖 Respuesta: {result['answer']}")
print(f"🔧 Tool called: {result.get('tool_called', False)}")

2026-01-22 14:34:20,424 - INFO - Search query: '¿Qué tiempo hará mañana para ir allí?...' returned 3 results
2026-01-22 14:34:20,425 - INFO - Retrieved 3 chunks for: ¿Qué tiempo hará mañana para ir allí?...
2026-01-22 14:34:21,855 - INFO - Completion with tools - finish_reason: tool_calls
2026-01-22 14:34:21,856 - INFO - Tool call requested: get_weather
2026-01-22 14:34:21,857 - INFO - Tool call arguments: {"date":"mañana"}
2026-01-22 14:34:21,858 - INFO - Weather request: date=2026-01-23, location=Tenerife, simulated=True
2026-01-22 14:34:21,858 - INFO - Weather response: Parcialmente nublado, 20°C
2026-01-22 14:34:21,859 - INFO - Tool result: {'date': '2026-01-23', 'location': 'Tenerife', 'temperature_high': 20, 'temperature_low': 14, 'condition': 'Parcialmente nublado', 'humidity': 56, 'wind_speed': 18, 'recommendation': 'Buen día para turismo. Lleva gafas de sol por si acaso.', 'simulated': True}
2026-01-22 14:34:24,416 - INFO - Completion received - tokens used: 1108
2026-01-22 14

👤 Turno 3: ¿Qué tiempo hará mañana para ir allí?
🤖 Respuesta: Mañana en Tenerife se espera un día **parcialmente nublado** con temperaturas que oscilarán entre los **14°C** y **20°C**. La humedad será del **56%** y habrá un viento de **18 km/h**.

Es un buen día para hacer turismo, así que te animo a que vayas a Punta de Teno. No olvides llevar gafas de sol por si acaso. ¡Espero que disfrutes de tu visita!
🔧 Tool called: True


In [11]:
# Mostrar historial
print("\n📜 Historial de conversación:")
for i, msg in enumerate(rag.get_history()):
    role = "👤" if msg["role"] == "user" else "🤖"
    content = msg["content"][:60] + "..." if len(msg["content"]) > 60 else msg["content"]
    print(f"   {i+1}. {role} {content}")


📜 Historial de conversación:
   1. 👤 ¿Qué playas hay en el norte de Tenerife?
   2. 🤖 En el norte de Tenerife, hay varias playas que puedes visita...
   3. 👤 ¿Cuál me recomiendas para ver el atardecer?
   4. 🤖 Te recomiendo la **Playa de Benijo** para disfrutar de un at...
   5. 👤 ¿Qué tiempo hará mañana para ir allí?
   6. 🤖 Mañana en Tenerife se espera un día **parcialmente nublado**...


---

## 5. Evaluación y Análisis

### 5.1 Batería de Tests

In [12]:
# Definir casos de prueba
test_cases = [
    # RAG - Información de lugares
    {"category": "RAG - Lugares", "query": "¿Cómo puedo subir al Teide?", "expected_tool": False},
    {"category": "RAG - Lugares", "query": "¿Qué puedo ver en Puerto de la Cruz?", "expected_tool": False},
    {"category": "RAG - Lugares", "query": "¿Dónde está el Drago Milenario?", "expected_tool": False},
    
    # RAG - Restaurantes
    {"category": "RAG - Restaurantes", "query": "¿Dónde puedo comer en el norte de Tenerife?", "expected_tool": False},
    {"category": "RAG - Restaurantes", "query": "¿Qué es un guachinche?", "expected_tool": False},
    
    # RAG - Playas
    {"category": "RAG - Playas", "query": "¿Qué playas tienen arena negra?", "expected_tool": False},
    {"category": "RAG - Playas", "query": "¿Cuál es la mejor playa para ver el atardecer?", "expected_tool": False},
    
    # Function Calling - Clima
    {"category": "Function - Clima", "query": "¿Qué tiempo hará mañana?", "expected_tool": True},
    {"category": "Function - Clima", "query": "¿Lloverá el fin de semana?", "expected_tool": True},
    {"category": "Function - Clima", "query": "¿Cuál es la temperatura actual en Tenerife?", "expected_tool": True},
    
    # Fuera de contexto
    {"category": "Fuera de contexto", "query": "¿Cuál es la capital de Francia?", "expected_tool": False},
    {"category": "Fuera de contexto", "query": "¿Cuánto es 2+2?", "expected_tool": False},
]

print(f"📋 Total de casos de prueba: {len(test_cases)}")

📋 Total de casos de prueba: 12


In [13]:
# Ejecutar tests
import time

results = []

for i, test in enumerate(test_cases):
    rag.clear_history()
    
    start_time = time.time()
    result = rag.query(test["query"], k=5)
    elapsed = time.time() - start_time
    
    tool_correct = result.get("tool_called", False) == test["expected_tool"]
    has_answer = len(result["answer"]) > 20
    
    results.append({
        "category": test["category"],
        "query": test["query"],
        "tool_called": result.get("tool_called", False),
        "tool_correct": tool_correct,
        "has_answer": has_answer,
        "answer_length": len(result["answer"]),
        "time": elapsed,
        "answer_preview": result["answer"][:100]
    })
    
    status = "✅" if tool_correct and has_answer else "⚠️"
    print(f"{status} [{i+1}/{len(test_cases)}] {test['category']}: {test['query'][:40]}... ({elapsed:.2f}s)")

print("\n✅ Tests completados")

2026-01-22 14:34:42,069 - INFO - Conversation history cleared
2026-01-22 14:34:42,441 - INFO - Search query: '¿Cómo puedo subir al Teide?...' returned 5 results
2026-01-22 14:34:42,442 - INFO - Retrieved 5 chunks for: ¿Cómo puedo subir al Teide?...
2026-01-22 14:34:50,682 - INFO - Completion with tools - finish_reason: stop
2026-01-22 14:34:50,684 - INFO - Generated answer (no tool call): 1535 chars
2026-01-22 14:34:50,685 - INFO - Conversation history cleared


✅ [1/12] RAG - Lugares: ¿Cómo puedo subir al Teide?... (8.61s)


2026-01-22 14:34:50,988 - INFO - Search query: '¿Qué puedo ver en Puerto de la Cruz?...' returned 5 results
2026-01-22 14:34:50,989 - INFO - Retrieved 5 chunks for: ¿Qué puedo ver en Puerto de la Cruz?...
2026-01-22 14:34:58,088 - INFO - Completion with tools - finish_reason: stop
2026-01-22 14:34:58,089 - INFO - Generated answer (no tool call): 1413 chars
2026-01-22 14:34:58,090 - INFO - Conversation history cleared


✅ [2/12] RAG - Lugares: ¿Qué puedo ver en Puerto de la Cruz?... (7.40s)


2026-01-22 14:34:58,619 - INFO - Search query: '¿Dónde está el Drago Milenario?...' returned 5 results
2026-01-22 14:34:58,620 - INFO - Retrieved 5 chunks for: ¿Dónde está el Drago Milenario?...
2026-01-22 14:35:01,583 - INFO - Completion with tools - finish_reason: stop
2026-01-22 14:35:01,584 - INFO - Generated answer (no tool call): 406 chars
2026-01-22 14:35:01,585 - INFO - Conversation history cleared


...

✅ [10/12] Function - Clima: ¿Cuál es la temperatura actual en Teneri... (3.59s)


2026-01-22 14:35:39,681 - INFO - Search query: '¿Cuál es la capital de Francia?...' returned 5 results
2026-01-22 14:35:39,682 - INFO - Retrieved 5 chunks for: ¿Cuál es la capital de Francia?...
2026-01-22 14:35:40,193 - INFO - Completion with tools - finish_reason: stop
2026-01-22 14:35:40,195 - INFO - Generated answer (no tool call): 25 chars
2026-01-22 14:35:40,195 - INFO - Conversation history cleared


✅ [11/12] Fuera de contexto: ¿Cuál es la capital de Francia?... (0.72s)


2026-01-22 14:35:40,498 - INFO - Search query: '¿Cuánto es 2+2?...' returned 5 results
2026-01-22 14:35:40,499 - INFO - Retrieved 5 chunks for: ¿Cuánto es 2+2?...
2026-01-22 14:35:40,958 - INFO - Completion with tools - finish_reason: stop
2026-01-22 14:35:40,959 - INFO - Generated answer (no tool call): 25 chars


✅ [12/12] Fuera de contexto: ¿Cuánto es 2+2?... (0.76s)

✅ Tests completados


### 5.2 Análisis de Resultados

In [15]:
# Calcular métricas
total_tests = len(results)
tool_accuracy = sum(1 for r in results if r["tool_correct"]) / total_tests * 100
answer_rate = sum(1 for r in results if r["has_answer"]) / total_tests * 100
avg_time = sum(r["time"] for r in results) / total_tests
avg_length = sum(r["answer_length"] for r in results) / total_tests

print("📊 MÉTRICAS GENERALES")
print("=" * 40)
print(f"Total de tests: {total_tests}")
print(f"Precisión tool calling: {tool_accuracy:.1f}%")
print(f"Tasa de respuesta: {answer_rate:.1f}%")
print(f"Tiempo promedio: {avg_time:.2f}s")
print(f"Longitud promedio respuesta: {avg_length:.0f} chars")

📊 MÉTRICAS GENERALES
Total de tests: 12
Precisión tool calling: 100.0%
Tasa de respuesta: 100.0%
Tiempo promedio: 4.91s
Longitud promedio respuesta: 688 chars


In [16]:
# Análisis por categoría
from collections import defaultdict

by_category = defaultdict(list)
for r in results:
    by_category[r["category"]].append(r)

print("📊 MÉTRICAS POR CATEGORÍA")
print("=" * 60)

for category, cat_results in by_category.items():
    n = len(cat_results)
    tool_acc = sum(1 for r in cat_results if r["tool_correct"]) / n * 100
    ans_rate = sum(1 for r in cat_results if r["has_answer"]) / n * 100
    avg_t = sum(r["time"] for r in cat_results) / n
    
    print(f"\n{category}:")
    print(f"   Tests: {n}")
    print(f"   Tool accuracy: {tool_acc:.0f}%")
    print(f"   Answer rate: {ans_rate:.0f}%")
    print(f"   Avg time: {avg_t:.2f}s")

📊 MÉTRICAS POR CATEGORÍA

RAG - Lugares:
   Tests: 3
   Tool accuracy: 100%
   Answer rate: 100%
   Avg time: 6.50s

RAG - Restaurantes:
   Tests: 2
   Tool accuracy: 100%
   Answer rate: 100%
   Avg time: 9.01s

RAG - Playas:
   Tests: 2
   Tool accuracy: 100%
   Answer rate: 100%
   Avg time: 4.15s

Function - Clima:
   Tests: 3
   Tool accuracy: 100%
   Answer rate: 100%
   Avg time: 3.86s

Fuera de contexto:
   Tests: 2
   Tool accuracy: 100%
   Answer rate: 100%
   Avg time: 0.74s


In [17]:
# Detalle de respuestas
print("📋 DETALLE DE RESPUESTAS")
print("=" * 60)

for r in results:
    tool_status = "🔧" if r["tool_called"] else "📚"
    print(f"\n{tool_status} {r['category']}")
    print(f"   Q: {r['query']}")
    print(f"   A: {r['answer_preview']}...")

📋 DETALLE DE RESPUESTAS

📚 RAG - Lugares
   Q: ¿Cómo puedo subir al Teide?
   A: Para subir al Teide, tienes varias opciones que te permitirán disfrutar de esta impresionante montañ...

📚 RAG - Lugares
   Q: ¿Qué puedo ver en Puerto de la Cruz?
   A: En Puerto de la Cruz, hay varias atracciones y lugares interesantes que puedes visitar. Aquí te dejo...

📚 RAG - Lugares
   Q: ¿Dónde está el Drago Milenario?
   A: El Drago Milenario se encuentra en Icod de los Vinos, en Tenerife. Este emblemático árbol es famoso ...

📚 RAG - Restaurantes
   Q: ¿Dónde puedo comer en el norte de Tenerife?
   A: ¡Claro! Aquí tienes algunas recomendaciones de lugares para comer en el norte de Tenerife:

### Zona...

📚 RAG - Restaurantes
   Q: ¿Qué es un guachinche?
   A: Un guachinche es un tipo de establecimiento típico de la zona norte de Tenerife, donde se ofrece com...

📚 RAG - Playas
   Q: ¿Qué playas tienen arena negra?
   A: En Tenerife, las playas que tienen arena negra son:

1. **Playa de Benijo**: 

### 5.3 Tests de Casos Límite

In [18]:
# Test: Fecha inválida para weather
print("🧪 TEST: Fecha muy lejana en el futuro")
rag.clear_history()
result = rag.query("¿Qué tiempo hará el 15 de marzo de 2030?")
print(f"   Respuesta: {result['answer'][:200]}...")
print(f"   Tool called: {result.get('tool_called', False)}")

2026-01-22 14:36:13,117 - INFO - Conversation history cleared


🧪 TEST: Fecha muy lejana en el futuro


2026-01-22 14:36:13,437 - INFO - Search query: '¿Qué tiempo hará el 15 de marzo de 2030?...' returned 3 results
2026-01-22 14:36:13,438 - INFO - Retrieved 3 chunks for: ¿Qué tiempo hará el 15 de marzo de 2030?...
2026-01-22 14:36:14,376 - INFO - Completion with tools - finish_reason: tool_calls
2026-01-22 14:36:14,377 - INFO - Tool call requested: get_weather
2026-01-22 14:36:14,378 - INFO - Tool call arguments: {"date":"2030-03-15"}
2026-01-22 14:36:14,379 - INFO - Weather request: date=2030-03-15, location=Tenerife, simulated=True
2026-01-22 14:36:14,379 - WARNING - Date too far in future: 2030-03-15
2026-01-22 14:36:14,380 - INFO - Tool result: {'error': True, 'message': 'Solo puedo proporcionar pronósticos hasta 7 días en el futuro.'}
2026-01-22 14:36:16,952 - INFO - Completion received - tokens used: 885
2026-01-22 14:36:16,954 - INFO - Generated answer after tool call: 265 chars


   Respuesta: Lo siento, pero no puedo proporcionar información sobre el tiempo para el 15 de marzo de 2030, ya que solo puedo ofrecer pronósticos hasta 7 días en el futuro. Si tienes planes más cercanos, estaré en...
   Tool called: True


In [19]:
# Test: Pregunta ambigua
print("🧪 TEST: Pregunta ambigua")
rag.clear_history()
result = rag.query("¿Qué me recomiendas?")
print(f"   Respuesta: {result['answer'][:200]}...")

2026-01-22 14:36:20,781 - INFO - Conversation history cleared


🧪 TEST: Pregunta ambigua


2026-01-22 14:36:21,043 - INFO - Search query: '¿Qué me recomiendas?...' returned 3 results
2026-01-22 14:36:21,044 - INFO - Retrieved 3 chunks for: ¿Qué me recomiendas?...
2026-01-22 14:36:28,419 - INFO - Completion with tools - finish_reason: stop
2026-01-22 14:36:28,420 - INFO - Generated answer (no tool call): 1350 chars


   Respuesta: ¡Claro! Aquí tienes algunas recomendaciones para disfrutar de Tenerife:

### Comida
1. **Arepera La Carajita**: Ubicada entre La Orotava y el Puerto de la Cruz, es famosa por tener las mejores arepas....


In [20]:
# Test: Pregunta en inglés
print("🧪 TEST: Pregunta en inglés")
rag.clear_history()
result = rag.query("What beaches can I visit in Tenerife?")
print(f"   Respuesta: {result['answer'][:200]}...")

2026-01-22 14:36:31,415 - INFO - Conversation history cleared


🧪 TEST: Pregunta en inglés


2026-01-22 14:36:31,982 - INFO - Search query: 'What beaches can I visit in Tenerife?...' returned 3 results
2026-01-22 14:36:31,984 - INFO - Retrieved 3 chunks for: What beaches can I visit in Tenerife?...
2026-01-22 14:36:40,200 - INFO - Completion with tools - finish_reason: stop
2026-01-22 14:36:40,201 - INFO - Generated answer (no tool call): 1061 chars


   Respuesta: En Tenerife, tienes varias playas maravillosas para visitar. Aquí te dejo algunas recomendaciones:

### Zona Sur
1. **El Médano / La Tejita**: 
   - Estas playas son muy típicas de Tenerife y ofrecen ...


In [21]:
# Test: Información no existente en la guía
print("🧪 TEST: Información no en la guía")
rag.clear_history()
result = rag.query("¿Cuánto cuesta la entrada al Loro Parque?")
print(f"   Respuesta: {result['answer'][:200]}...")

2026-01-22 14:36:42,777 - INFO - Conversation history cleared


🧪 TEST: Información no en la guía


2026-01-22 14:36:43,024 - INFO - Search query: '¿Cuánto cuesta la entrada al Loro Parque?...' returned 3 results
2026-01-22 14:36:43,025 - INFO - Retrieved 3 chunks for: ¿Cuánto cuesta la entrada al Loro Parque?...
2026-01-22 14:36:43,575 - INFO - Completion with tools - finish_reason: stop
2026-01-22 14:36:43,576 - INFO - Generated answer (no tool call): 25 chars


   Respuesta: No tengo esa información....


### 5.4 Resumen de Evaluación

In [ ]:
print("="*60)
print("📊 RESUMEN DE EVALUACIÓN")
print("="*60)

print("""
✅ FUNCIONALIDADES IMPLEMENTADAS:
   - RAG con búsqueda semántica sobre guía de Tenerife
   - Diálogo multiturno con memoria de conversación
   - Function calling para pronóstico del tiempo
   - Manejo de errores y validación de datos

📈 FORTALEZAS:
   - Respuestas precisas basadas en la guía turística
   - Detección correcta de intención (RAG vs Weather)
   - Mantiene contexto en conversaciones multiturno
   - Tiempo de respuesta aceptable (~2-3s)

⚠️ LIMITACIONES:
   - Solo información contenida en la guía PDF
   - Clima simulado (no real)
   - Sin soporte para imágenes o multimedia
   - Dependencia de calidad de chunks para retrieval

🔮 MEJORAS FUTURAS:
   - Integrar API de clima real
   - Añadir más funciones (reservas, transporte)
   - Evaluación con métricas de NLP (BLEU, ROUGE)
""")

---

## 6. Conclusiones

Este proyecto implementa un asistente turístico conversacional completo que combina:

1. **RAG (Retrieval-Augmented Generation)**: Permite responder preguntas específicas sobre Tenerife usando una guía turística como fuente de verdad, evitando alucinaciones.

2. **Diálogo Multiturno**: El asistente mantiene el contexto de la conversación, permitiendo preguntas de seguimiento naturales.

3. **Function Calling**: Integración con servicios externos (pronóstico del tiempo) que extiende las capacidades del asistente más allá del conocimiento estático.

La arquitectura modular del código permite fácil extensión y mantenimiento, con separación clara de responsabilidades entre los diferentes componentes.